In [16]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
import glob

import sys
sys.path.append('cropping')
import load_read_name_extractor as lrne


In [17]:
data_paths_auto = glob.glob("Features/*npy")
size = int(len(data_paths_auto)/2)
features_paths = data_paths_auto[0:size]
names_paths = data_paths_auto[size:len(data_paths_auto)]

In [18]:
features_paths


['Features\\hog_features_(64, 128, 3)_cells_(10, 10)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(12, 12)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(14, 14)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(16, 16)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block

In [19]:
names_paths

['Features\\img_names_(64, 128, 3)_cells_(10, 10)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(12, 12)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(14, 14)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(16, 16)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_n

In [20]:
features_paths = [
'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy']

In [21]:
names_paths = [
  'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy']

In [22]:
features_load = np.load(names_paths[0], allow_pickle=True)
features_load[1000].shape

()

In [23]:
np.load(names_paths[0], allow_pickle=True) == np.load(names_paths[1], allow_pickle=True)

array([ True,  True,  True, ...,  True,  True,  True])

In [24]:
features_load[3000].shape

()

In [25]:
def save_results(results, path):
    path = path.replace("Features\\", "")
    path = "results/" + path.replace(".npy", ".txt")
    with open(path, 'a') as f:
        f.write(results)
        f.write("\n")
    print("Results Saved Succesfull")

In [26]:
def load_labels(bin_size):
    labels = get_lodging_scores.get_labels(bin_size)
    #COnverts into set (no dublicates in set)
    my_set = set(map(tuple, labels))
    # convert set back to list of arrays
    labels = list(map(np.array, my_set))
    return labels

In [27]:
def partition_data(data_frame, percentage):
    partition = int(len(data_frame)*percentage/100)
    x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
    y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
    print("Data Partioned Succesfull")
    return x_train, x_test, y_train, y_test

In [28]:
def save_model(clf, filename):
    filename = filename.replace("Features\\", "")
    filename = "model/" + filename.replace(".npy", ".sav")
    pickle.dump(clf, open(filename, 'wb'))
    print("Model Saved Succesfull")

In [29]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights = None):
    for i in range (len(features_paths)):
        #laod features and names
        features_load = np.load(features_paths[i], allow_pickle=True)
        names_load = np.load(names_paths[i], allow_pickle=True)

        print(1)
        #load labels
        labels = load_labels(bin_size)

        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)
        names_load = None
        print(2)
        #match labels with feature names
        data_frame = svm_general.match_pic_label_to_names(features_load, labels, names)
        print(3)
        features_load = None
        labels = None
        names = None
        print(data_frame)
        #shuffle data and convert to array
        np.random.shuffle(data_frame)

        #partition data
        percentage = 80
        data_frame = np.array(data_frame)
        x_train, x_test, y_train, y_test = partition_data(data_frame, percentage)
        data_frame = None
        features_aar = None

        #create svm
        clf = svm.SVC(kernel=svm_kernel, class_weight=class_weights)
        #fit model with training data
        clf.fit(x_train,y_train)
        #predict data
        y_pred = clf.predict(x_test)

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        print(features_paths[i])
        print(len(features_paths))
        path = "SVMC_" + "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + str(class_weights) + "_" + features_paths[i]
        save_model(clf, path)

        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        save_results(results, path)

In [30]:
run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf")

1
2
3
[array(['0.16200769801102333', '0.15171845220727487',
       '0.008096216102272243', ..., '0.15804435524212174',
       '0.05182140916764998', '3'], dtype='<U32'), array(['0.19430890001740359', '0.05338149837943895', '0.0705515186663742',
       ..., '0.06877516178297406', '0.13706853187519993', '0'],
      dtype='<U32'), array(['0.12802445061188306', '0.1002308064620404', '0.07519245641774207',
       ..., '0.25108704738081783', '0.1754203409185182', '1'],
      dtype='<U32'), array(['0.13907409525710723', '0.08757440689445738',
       '0.027158587294137754', ..., '0.21646016105311058',
       '0.08045809294952642', '3'], dtype='<U32'), array(['0.1412292798954866', '0.03459501756429004', '0.12965858627199875',
       ..., '0.0642165619199596', '0.1442425945711035', '0'], dtype='<U32'), array(['0.2925972948845922', '0.09242421461720296', '0.11531424701623448',
       ..., '0.10285637002170385', '0.2083081301212102', '1'],
      dtype='<U32'), array(['0.05768127903900486', '0.0292

In [ ]:
print(features_paths[0])
print(len(features_paths))

Features\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy
1
